## Loading the Data
Let's import the libraries we'll use.

In [10]:
# database
from sqlalchemy import create_engine
from sqlalchemy import inspect

# data processing
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import pickle

# geographical data
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame
import shapefile
from shapely.geometry import Point # Point class
from shapely.geometry import shape # shape() is a function to convert geo objects through the interface
import shapely

import functions as fn

## Import and Clean Data

In [11]:
engine = create_engine('sqlite:////Users/NeilenBenvegnu/Desktop/METIS/NBM_Engineering_Student/project/storm_events.db')
insp = inspect(engine)
print(insp.get_table_names())

['details', 'fatalities', 'locations']


In [12]:
data = pd.read_sql('SELECT * FROM details;', engine)

In [13]:
storm_data = data.copy()

# transform column names to lower case
storm_data.columns = storm_data.columns.str.lower()
pd.set_option('max_columns', None)

# restrict to just US States, exclude marine events
storm_data = storm_data[storm_data['cz_type'] != 'M']
storm_data = storm_data[storm_data['state_fips'] < 57]
storm_data['state'] = storm_data['state'].str.title()
storm_data['cz_name'] = storm_data['cz_name'].str.title()


# format begin_date and end_date columns 
storm_data.insert(0, 'begin_datetime', \
                  pd.to_datetime(storm_data['begin_yearmonth'].astype(str) + \
                                 storm_data['begin_day'].astype(str), format='%Y%m%d'))

storm_data.insert(1, 'end_datetime', \
                  pd.to_datetime(storm_data['end_yearmonth'].astype(str) + \
                                 storm_data['end_day'].astype(str), format='%Y%m%d'))

# drop extra columns
cols_to_drop = ['begin_yearmonth', 'begin_day','begin_time', \
                'end_yearmonth', 'end_day','end_time', \
                'episode_id','month_name', 'begin_lat', 'begin_lon', 'end_lat', 'end_lon',\
                'wfo', 'begin_date_time','cz_timezone', 'end_date_time', 'source', \
                'magnitude', 'magnitude_type', 'flood_cause', 'category', 'tor_f_scale', \
                'tor_length', 'tor_width', 'tor_other_wfo', 'tor_other_cz_fips', 'tor_other_cz_name', \
                'tor_other_cz_state', 'begin_range', 'begin_azimuth', 'begin_location','end_range', \
                'end_azimuth', 'end_location', 'episode_narrative', 'event_narrative', 'data_source']

storm_data.drop(labels=cols_to_drop, axis=1, inplace=True)

# consolidate, format injuries and deaths columns
storm_data.insert(8, 'injuries', storm_data['injuries_direct'] + storm_data['injuries_indirect'])
storm_data.drop(labels=['injuries_direct', 'injuries_indirect'], axis=1, inplace=True)

storm_data.insert(9, 'deaths', storm_data['deaths_direct'] + storm_data['deaths_indirect'])
storm_data.drop(labels=['deaths_direct', 'deaths_indirect'], axis=1, inplace=True)

storm_data['damage_property'] = storm_data['damage_property'].fillna(0).apply(fn.format_money)
storm_data['damage_crops'] = storm_data['damage_crops'].fillna(0).apply(fn.format_money)

# combine state, county FIPS into single FIP code
storm_data['state_fips'] = storm_data['state_fips'].astype(int).astype(str).str.zfill(2)
storm_data['cz_fips'] = storm_data['cz_fips'].astype(str).str.zfill(3)
storm_data['fips'] = storm_data['state_fips'] + storm_data['cz_fips']
storm_data.drop(labels=['state_fips', 'cz_fips'], axis=1, inplace=True)

storm_data.head()

,begin_datetime,end_datetime,event_id,state,year,event_type,cz_type,injuries,deaths,cz_name,damage_property,damage_crops,fips
0,1998-11-10,1998-11-10,5670481,Ohio,1998,Thunderstorm Wind,C,0,0,Delaware,3000.0,0.0,39041
1,1998-12-31,1998-12-31,5673776,New Jersey,1998,Drought,Z,0,0,Gloucester,0.0,0.0,34017
2,1998-12-23,1998-12-24,5673835,Virginia,1998,Ice Storm,Z,0,0,Grayson,0.0,0.0,51015
3,1998-10-01,1998-10-01,5667315,New Mexico,1998,Hail,C,0,0,Rio Arriba,0.0,0.0,35039
4,1998-09-29,1998-09-29,5664635,Georgia,1998,Thunderstorm Wind,C,0,0,Macon,5000.0,0.0,13193


## Convert Forecast Zones to County FIPS

We have a problem with our cleaned databse. Storm events logged as occuring in 'Forecast Zones' (`cz_type` = Z) won't be appropriately graphed in their counties because their FIP codes are not recognized. We want to transform the FIP codes of Zone events so that they reflect the nearest, most relevant counties geographically.


In [14]:
# aggregator to hold Zone-FIPS conversion dataframes for each state
zone_fips_agg = []

# iterate through states
states_list = storm_data['state'].unique().tolist()

for state in states_list:
    
    # create dataframe with state's County names, FIPS
    state_df = storm_data[(storm_data['state'] == state) & (storm_data['cz_type'] == 'C')]
    county_to_fips = state_df[['cz_name', 'fips']].groupby('cz_name').agg({'fips': 'max'}).reset_index()
    
    # create dataframe with state's Zone names, County Names
    zone_to_county = fn.to_county_mapper(storm_data, state) 
    zone_to_county = pd.DataFrame(zone_to_county, columns = ['zone', 'cz_name'])
    
    # merge into dataframe with state's Zone names, FIPS
    # for Zones with more than one county, will FIPS column will contain a list of multiple FIPS
    zone_fips_df = pd.merge(zone_to_county, county_to_fips, left_on = ['cz_name'], right_on = ['cz_name'], how='left')
    zone_to_fips = zone_fips_df.groupby('zone').agg({'fips':lambda x: list(x)}).reset_index()
    
    # aggregate 51 state-level dataframes to hold Zone, FIPS conversions in a list
    zone_to_fips['state'] = state
    zone_fips_agg.append(zone_to_fips)

# convert list of state-level Zone, FIPS dataframes into a single dataframe
zone_fips_state = pd.DataFrame()
zone_fips_state = pd.concat([df for df in zone_fips_agg])
zone_fips_state.rename(columns = {'zone':'cz_name'},inplace = True)
zone_fips_state

,cz_name,fips,state
0,Adams,[39001],Ohio
1,Allen,[39003],Ohio
2,Ashland,[39005],Ohio
3,Ashtabula,[39007],Ohio
4,Ashtabula Lakeshore,"[39007, 39085]",Ohio
...,...,...,...
55,Valley/Adams/N Gem/N Washington,"[16001, 16087, 16003, 16085, 16045]",Idaho
56,W Benewah/W Latah,"[16057, 16009]",Idaho
57,W Franklin,[16041],Idaho
58,Western Lemhi County,[16059],Idaho


In [15]:
new_storm_data = pd.merge(storm_data,zone_fips_state,left_on = ['state','cz_name'],right_on=['state','cz_name'],how = 'left')
new_storm_data = new_storm_data.explode('fips_y')

repeat_rows = new_storm_data[['event_id', 'fips_x']].groupby('event_id').agg({'fips_x': 'count'}).reset_index()
repeat_rows.rename(columns = {'fips_x':'num_affected_fips'},inplace = True)
new_storm_data = pd.merge(new_storm_data,repeat_rows,left_on = ['event_id'],right_on=['event_id'],how = 'left')

numeric_cols = ['injuries', 'deaths', 'damage_property', 'damage_crops']
new_storm_data[numeric_cols] = new_storm_data[numeric_cols].div(new_storm_data['num_affected_fips'], axis=0)

In [16]:
new_storm_data.head()

,begin_datetime,end_datetime,event_id,state,year,event_type,cz_type,injuries,deaths,cz_name,damage_property,damage_crops,fips_x,fips_y,num_affected_fips
0,1998-11-10,1998-11-10,5670481,Ohio,1998,Thunderstorm Wind,C,0.0,0.0,Delaware,3000.0,0.0,39041,39041,1
1,1998-12-31,1998-12-31,5673776,New Jersey,1998,Drought,Z,0.0,0.0,Gloucester,0.0,0.0,34017,34015,1
2,1998-12-23,1998-12-24,5673835,Virginia,1998,Ice Storm,Z,0.0,0.0,Grayson,0.0,0.0,51015,51077,1
3,1998-10-01,1998-10-01,5667315,New Mexico,1998,Hail,C,0.0,0.0,Rio Arriba,0.0,0.0,35039,NaN,1
4,1998-09-29,1998-09-29,5664635,Georgia,1998,Thunderstorm Wind,C,0.0,0.0,Macon,5000.0,0.0,13193,13193,1


In [17]:
#make new final fip column
#if cz_type == Z and fips_y.notnull(), use fips_y
#else, use fips_x
fips_url = 'https://raw.githubusercontent.com/kjhealy/fips-codes/master/state_and_county_fips_master.csv'
fips_df = pd.read_csv(fips_url)[2:]
fips_list = fips_df['fips'].astype(str).str.zfill(5).tolist()

is_zone = new_storm_data['cz_type'] == 'Z'
yfips_notnull = new_storm_data['fips_y'].notnull()
yfips_required = ~new_storm_data['fips_x'].isin(fips_list)
new_storm_data['fips'] = np.where(is_zone & yfips_notnull & yfips_required, new_storm_data['fips_y'], new_storm_data['fips_x'])

storm_data_final = new_storm_data.drop(columns=['begin_datetime', 'end_datetime', 'event_id', 'cz_type', 'fips_x', 'fips_y', 'num_affected_fips'])
storm_data_final.rename(columns = {'cz_name' : 'county_name'}, inplace=True)

storm_data_final['event_type'] = storm_data_final['event_type'].map(fn.events_dict)
storm_data_final

,state,year,event_type,injuries,deaths,county_name,damage_property,damage_crops,fips
0,Ohio,1998,Extreme Wind,0.0,0.0,Delaware,3000.0,0.0,39041
1,New Jersey,1998,Drought,0.0,0.0,Gloucester,0.0,0.0,34017
2,Virginia,1998,Winter Storm,0.0,0.0,Grayson,0.0,0.0,51015
3,New Mexico,1998,Winter Storm,0.0,0.0,Rio Arriba,0.0,0.0,35039
4,Georgia,1998,Extreme Wind,0.0,0.0,Macon,5000.0,0.0,13193
...,...,...,...,...,...,...,...,...,...
1121930,Michigan,1997,Winter Storm,0.0,0.0,Marquette,0.0,0.0,26103
1121931,Arizona,1997,Flooding,1.0,2.0,Coconino,0.0,0.0,04005
1121932,Arizona,1997,Winter Storm,0.0,0.0,Gila,0.0,0.0,04007
1121933,Arizona,1997,Tornado,0.0,0.0,Coconino,0.0,0.0,04005


In [19]:
storm_data_final.to_pickle("storm_data_final.pkl")


In [21]:
storm_data_final.state.unique().tolist()

['Ohio',
 'New Jersey',
 'Virginia',
 'New Mexico',
 'Georgia',
 'Texas',
 'Mississippi',
 'Oklahoma',
 'Missouri',
 'Kentucky',
 'Wisconsin',
 'Michigan',
 'Colorado',
 'Alaska',
 'Illinois',
 'Nevada',
 'Arizona',
 'West Virginia',
 'Minnesota',
 'Connecticut',
 'Pennsylvania',
 'Maine',
 'New Hampshire',
 'Wyoming',
 'Hawaii',
 'Maryland',
 'Iowa',
 'Vermont',
 'California',
 'Nebraska',
 'Massachusetts',
 'Utah',
 'New York',
 'North Carolina',
 'South Dakota',
 'Arkansas',
 'Alabama',
 'Montana',
 'Washington',
 'Oregon',
 'Florida',
 'Indiana',
 'South Carolina',
 'Tennessee',
 'Kansas',
 'District Of Columbia',
 'Louisiana',
 'North Dakota',
 'Delaware',
 'Rhode Island',
 'Idaho']